# Import Library Yang Dibutuhkan

In [1]:
from scipy.signal import kaiserord, lfilter, firwin, freqz
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import signal
import collections
from numpy import savetxt
from scipy.stats import entropy
from math import log, e
import os, sys
from scipy.signal import butter, iirnotch, lfilter
from scipy.stats import norm, kurtosis
from scipy.stats import skew    
import neurokit2 as nk


C:\Users\helmi.ruslan\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\helmi.ruslan\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar'

In [2]:
def shannon_entropy(data):
    bases = collections.Counter([tmp_base for tmp_base in data])
    # define distribution
    dist = [x/sum(bases.values()) for x in bases.values()]
 
    # use scipy to calculate entropy
    entropy_value = entropy(dist, base=2)
 
    return entropy_value



# Baca Dataset, Denoising serta Feature Extraction

In [3]:
J=0 # jumlah file
directory_path = 'dataset/sehat'
for iter in range(0,2):
    for x in os.listdir(directory_path):
        if not x.lower().endswith('.csv'):
            continue
        J=J+1
    directory_path = 'dataset/pasien'
n = J #jumlah file
m = 8
FEAT = [] #bakal jadi Feature.csv
for i in range(n): 
    FEAT.append([0] * m) #mengisi dengan angka 0 semua 
directory_path = 'dataset/sehat'
J=-1
K=0
FEAT
for iter in range(0,2):
    for x in os.listdir(directory_path):
        if not x.lower().endswith('.csv'):
            continue
        full_file_path = directory_path  +   '/'   + x
        J=J+1
        print ('Using file', full_file_path)
        try:
            dataraw = pd.read_csv(full_file_path,index_col='Timestamp', parse_dates=['Timestamp'])
            dataset = pd.DataFrame(dataraw['Value']) #ambil kolom value dari setiap file
        except:
            dataraw = pd.read_csv(full_file_path,index_col='timestamp', parse_dates=['timestamp'])
            dataset = pd.DataFrame(dataraw['values'])
        x1=np.array(dataset)  #ubah jadi array, namanya x1
        Dat=[]
        Dat=[0 for i in range(x1.size)] #bikin array kosong isinya 0 semua sepajang array x1
        for i in range(0,x1.size-1):
            Dat[i]=max(x1[i]) #why pakai max?
        
        fs = 110
        cutoff_low = 3
        powerline=60

        # FIR Filter
        # The Nyquist rate of the signal.
        nyq_rate = fs / 2.0
        # The desired width of the transition from pass to stop,
        # relative to the Nyquist rate.  We'll design the filter
        # with a 5 Hz transition width.
        width = 5.0/nyq_rate
        # The desired attenuation in the stop band, in dB.
        # Compute the order and Kaiser parameter for the FIR filter.
        N, beta = kaiserord(powerline, width)
        # The cutoff frequency of the filter.
        cutoff_hz = cutoff_low
        # Use firwin with a Kaiser window to create a lowpass FIR filter.
        taps = firwin(N, cutoff_hz/nyq_rate, window=('kaiser', beta))
        # Use lfilter to filter x with the FIR filter.
        y_filtered = lfilter(taps, 1.0, Dat)
       
        # FEATURE EXTRACTION
        try:

            # Shannon entropy
            ESH=shannon_entropy(y_filtered)
            FEAT[J][0] = ESH
            print('atas',FEAT[J][0])
            

            # MAD
            series = pd.Series(y_filtered)
            MAD = series.mad()
            # Kurtosis
            KURT = kurtosis(y_filtered)
            # Skewness
            SKEW = skew(y_filtered)
            print('atas',SKEW)

            #vlf,lf,hf
            info = nk.ppg_findpeaks(y_filtered)
            peak = info["PPG_Peaks"]
            hrv_freq = nk.hrv_frequency(peak, sampling_rate=39, normalize=True)
            VLF=hrv_freq['HRV_VLF'].values[0]
            LF=hrv_freq['HRV_LF'].values[0]
            HF=hrv_freq['HRV_HF'].values[0]
            FEAT[J][0] = ESH
            FEAT[J][1] = MAD
            FEAT[J][2] = KURT
            FEAT[J][3] = SKEW
            FEAT[J][4] = VLF
            FEAT[J][5] = LF
            FEAT[J][6] = HF
            FEAT[J][7] = K
            print(FEAT[J][0])
            
            
        except:
            J=J-1
    directory_path = 'dataset/pasien'
    K=1


Using file dataset/sehat/sehat1-nizar.csv
atas 12.679919878518422
atas -0.012679715959528211
12.679919878518422
Using file dataset/sehat/sehat10-bagas.csv
atas 12.66755510704113
atas -0.7528800318200992
12.66755510704113
Using file dataset/sehat/sehat11-fian.csv
atas 12.674412981837069
atas -0.06518578563063887
12.674412981837069
Using file dataset/sehat/sehat12-faizalaryo.csv
atas 12.6686634231753
atas -0.15255966216526773
12.6686634231753
Using file dataset/sehat/sehat13-dani.csv
atas 12.673750739438056
atas -0.5674472159487927
12.673750739438056
Using file dataset/sehat/sehat14 -pradana.csv
atas 12.683871868622601
atas -0.5672560176668867
12.683871868622601
Using file dataset/sehat/sehat15-rifkun.csv
atas 12.681238411777807
atas 0.011938424974833802
12.681238411777807
Using file dataset/sehat/sehat16-Nirel.csv
atas 12.759264045188791
atas -2.3206446755598833
12.759264045188791
Using file dataset/sehat/sehat17-Sesil.csv
atas 12.963437998346288
atas -2.910367209994084
12.9634379983462

# Export Hasil Ekstraksi Fitur 

In [4]:
import csv
#bikin csv
with open("Feature3.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(FEAT)

# with open("Feature2.csv", "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerows(FEAT2)

# Memberi Label Pada Dataset

In [6]:
dataset = pd.read_csv('Feature3.csv', names=['ESH', 'MAD', 'SKEW','KURT','VLF','LF','HF','Label'])

label = dataset['Label']
dataset = dataset.drop(columns='Label')

# View Dataset

In [7]:
dataset

,ESH,MAD,SKEW,KURT,VLF,LF,HF
0,12.679920,0.131682,0.214764,-0.012680,0.002333,0.000064,8.816926e-07
1,12.667555,0.143808,-0.023274,-0.752880,0.005205,0.000109,1.310033e-06
2,12.674413,0.118264,0.103585,-0.065186,0.006036,0.001193,3.904563e-06
3,12.668663,0.135634,-0.315434,-0.152560,0.004249,0.000512,4.543542e-07
4,12.673751,0.140251,0.364347,-0.567447,0.002141,0.000079,1.251686e-06
5,12.683872,0.146443,0.020874,-0.567256,0.000932,0.000015,1.896322e-08
6,12.681238,0.167376,-0.618047,0.011938,0.005110,0.000204,1.340477e-06
7,12.759264,0.061090,12.278461,-2.320645,0.010483,0.003694,1.611975e-06
8,12.963438,0.050025,15.738303,-2.910367,0.004100,0.000733,3.439706e-07
9,12.963438,0.050025,15.738303,-2.910367,0.004100,0.000733,3.439706e-07


# FIll Nilai kosong Pada fitur

In [88]:
dataset['VLF'] = dataset['VLF'].fillna(dataset['VLF'].mean())


In [87]:
dataset.isnull().sum()

ESH     0
MAD     0
SKEW    0
KURT    0
VLF     1
LF      0
HF      0
dtype: int64

# Tahapan Klasifikasi

In [439]:
# MACHINE LEARNING
# SPLIT DATA 70% TRAIN, 30% DATA TEST
from sklearn.model_selection import train_test_split

x = random.randint(100)
print(x)

53,24
X_train, X_test, y_train, y_test = train_test_split(dataset, label, test_size = 0.30,random_state=24)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
# dc29

print('DATA TRAIN')
print(X_train.shape)
print('LABEL TRAIN')
print(y_train.shape)
print('DATA TEST')
print(X_test.shape)
print('LABEL TEST')
print(y_test.shape)


# Import the classifier from sklearn
from sklearn.tree import DecisionTreeClassifier

# TODO: Define the classifier, and fit it to the data
model = DecisionTreeClassifier(random_state=20)
model.fit(X_train, y_train) 
y_test_pred = model.predict(X_test)

print('HASIL PREDIKSI')
print(y_test_pred)
y_train_pred = model.predict(X_train)

from sklearn.metrics import accuracy_score
train_accuracy = accuracy_score(y_train, y_train_pred)
cm2 = confusion_matrix(y_test, y_test_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
print('The training accuracy is', train_accuracy)
print('The test accuracy is', test_accuracy)


93
DATA TRAIN
(40, 7)
LABEL TRAIN
(40,)
DATA TEST
(18, 7)
LABEL TEST
(18,)
HASIL PREDIKSI
[0 1 1 0 0 0 1 0 1 0 1 0 0 1 1 0 0 1]
The training accuracy is 1.0
The test accuracy is 0.9444444444444444
HASIL PREDIKSI
[1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 1 0 1]
Akurasi SVM 50.0  %


C:\Users\helmi.ruslan\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)


In [440]:
TP = cm2[1][1]
TN = cm2[0][0]
FP = cm2[0][1]
FN = cm2[1][0]


In [437]:
TP = cm[1][1]
TN = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]


# Hasil Sensitiviti, Spesifisitas

In [441]:
# calculate accuracy
conf_accuracy = (float (TP+TN) / float(TP + TN + FP + FN))

# calculate mis-classification
conf_misclassification = 1- conf_accuracy

# calculate the sensitivity
conf_sensitivity = (TP / float(TP + FN))
# calculate the specificity
conf_specificity = (TN / float(TN + FP))

# calculate precision
conf_precision = (TN / float(TN + FP))
print(conf_sensitivity)
print(conf_specificity)

1.0
0.9090909090909091
